1. Transformer와 비교해 변경이 필요한 부분
- GPT-1에는 인코더가 포함되어있지 않다. (인코더 입출력 등 인코더와 관련된 부분 삭제)
    
    `class EncoderLayer, class Encoder, class Transformer 참고`


- Decoder 부분에서 Encoder-Decoder Attention 부분이 없다. (Decoder의 첫번째 Attention에서 두번째 Attention을 건너뛰어야됨)
    
    `class DecoderLayer, class Decoder 참고`


- 입력 데이터의 형식에서 Q와 A를 하나의 시퀀스로 넣는다.

    `class ChatBotDataset 참고`

- Positional Encoding 방식이 고정된 위치 인코딩에서 학습 가능한 위치 임베딩으로 바뀐다.

2. 모델의 입력 형태에 맞게 전처리를 수행하였다.
- GPT-1 모델은 디코더만 존재하기 때문에 인코더의 인풋은 고려하지 않는다.
- 디코더의 input : 데이터 전처리 과정에서 논문에서 요구하듯이 s Q + $[sep] + A /s 형태로 입력하도록 하였다.

    `class ChatBotDataset 참고`

3. 모델의 입력 블럭을 GPT 논문에 기반하여 수정하였다.
- 모델의 iput이 정상적으로 구성되었는지, 데이터에 위치 정보를 추가하는 과정 구현 확인

    `class Transformer, class Decoder, class DecoderLayer , class GPTPosionalEncoding 참고`

4. GPT 모델을 정상적으로 구성하였다.

|![model.summary](model.summary.png)|![model.fit](model.fit.png)|

5. 입력에 따른 출력이 생성되었다.

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import sentencepiece as spm
import pandas as pd
import math
import os
import re
import urllib.request
import zipfile
import numpy as np
import matplotlib.pyplot as plt

STEP1. 데이터 수집하기

STEP2. 데이터 전처리하기
- 영어 데이터와는 전혀 다른 데이터인 만큼 영어 데이터에 사용했던 전처리와 일부 동일한 전처리도 필요하겠지만 전체적으로는 다른 전처리를 수행해야 할 수도 있습니다.

In [26]:
path_to_dataset = "C:/Users/Choi/Desktop/ChatbotData.csv"
MAX_SAMPLES = 300000

# 전처리 함수
def preprocess_sentence(sentence):
  # 입력받은 sentence를 소문자로 변경하고 양쪽 공백을 제거
  sentence = sentence.lower().strip()

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  sentence = re.sub(r"[^ㄱ-ㅎ가-힣0-9?.!,]+", " ", sentence)
  sentence = sentence.strip()
  return sentence

sample_sentence = "Hello? 안녕하세요! 저는 학생입니다.😊, 만나서 반갑습니다.!"
preprocessed_sentence = preprocess_sentence(sample_sentence)
print(preprocessed_sentence)

? 안녕하세요 ! 저는 학생입니다 . , 만나서 반갑습니다 . !


In [27]:
# 데이터를 로드하는 동시에 전처리 함수를 호출하여 질문과 답변의 쌍을 전처리
def read_korean_chatbot_data(path_to_csv, max_samples=300000):
    print(f"데이터 로딩: {path_to_csv}")
    try:
        # ChatbotData.csv 파일은 'utf-8' 인코딩인 경우가 많습니다.
        df = pd.read_csv(path_to_csv, encoding='utf-8')
    except UnicodeDecodeError:
        # 만약 utf-8 실패 시 'cp949' 시도
        df = pd.read_csv(path_to_csv, encoding='cp949')

    pairs = []

    for index, row in df.iterrows():
        q_text = preprocess_sentence(row['Q'])
        a_text = preprocess_sentence(row['A'])
        
        # 전처리 후 비어있지 않은 쌍만 추가
        if q_text and a_text:
            pairs.append((q_text, a_text))
        
        if len(pairs) >= max_samples:
            print(f"최대 {max_samples}개 샘플 로딩 완료.")
            break
            
    return pairs

#로드한 데이터의 샘플 수 확인
pairs = read_korean_chatbot_data(path_to_dataset, max_samples=MAX_SAMPLES)

print('전체 샘플 수 :', len(pairs))

데이터 로딩: C:/Users/Choi/Desktop/ChatbotData.csv
전체 샘플 수 : 11823


STEP3. SentencePiece 사용하기
- 한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다. 하지만 여기서는 형태소 분석기가 아닌 위 실습에서 사용했던 서브워드 토크나이저인 SentencePiece를 그대로 사용해보세요.

In [28]:
#1 Toklenizer 학습하기
    #위에서 만든 pair를 텍스트 파일에 저장
corpus_file = "korean_chatbot_corpus.txt"
with open(corpus_file, 'w', encoding='utf-8') as f:
    for q, a in pairs:
        f.write(q + "\n")
        f.write(a + "\n")

#질문-답변 쌍이 담긴 파일을 이용해 모델 훈련하고 파일 생성
spm.SentencePieceTrainer.Train(
    input=corpus_file,
    model_prefix="spm_korean_chatbot",
    vocab_size=8000,
    character_coverage=1.0,
    model_type="bpe",
    max_sentence_length=999999,
    bos_id=1,  # <s> (Beginning of Sentence) 설정
    eos_id=2,  # </s> (End of Sentence) 설정
    pad_id=0,  # Padding ID 설정
    unk_id=3   # Unknown Token ID 설정
)

#학습 시킨 토크나이저를 불러와 테스트
sp = spm.SentencePieceProcessor()
sp.Load("spm_korean_chatbot.model")

# 예제 문장
sentence = "나는 센텐스피스로 파이토치를 배우고 있어요"

sentence = preprocess_sentence(sentence)
print("전처리 후의 문장:", sentence)

# 1. 토크나이징 (subword 단위로 분할)
tokens = sp.encode(sentence, out_type=str)
print("Tokenized:", tokens)

# 2. 인코딩 (서브워드를 정수 ID로 변환)
encoded = sp.encode(sentence, out_type=int)
print("Encoded:", encoded)

# 3. 디코딩 (정수 ID → 원본 문장 복원)
decoded = sp.decode(encoded)
print("Decoded:", decoded)

전처리 후의 문장: 나는 센텐스피스로 파이토치를 배우고 있어요
Tokenized: ['▁나는', '▁센', '텐', '스피', '스로', '▁파', '이', '토', '치', '를', '▁배우', '고', '▁있어요']
Encoded: [688, 1858, 7136, 3600, 861, 860, 6788, 7406, 6960, 6845, 1072, 6797, 115]
Decoded: 나는 센텐스피스로 파이토치를 배우고 있어요


In [29]:
#2 Dataset 구현하기
class ChatbotDataset(Dataset):
    def __init__(self, pairs, sp, max_length=40):
        super().__init__()
        self.sp = sp
        self.max_length = max_length
        self.data = []

        pad_id = sp.pad_id() # 0
        bos_id = sp.bos_id() # 1
        eos_id = sp.eos_id() # 2

        for q_text, a_text in pairs:
            # Q와 A를 하나의 시퀀스로 만들기
            combined = f"{q_text} [SEP] {a_text}"
            ids = sp.EncodeAsIds(combined)

            # 시작과 끝 추가
            tokens = [bos_id] + ids + [eos_id]

            # 3) 길이 제한
            if len(tokens) > max_length:
                continue

            # 4) 고정 길이 패딩
            pad = [pad_id] * (max_length - len(tokens))
            tokens = tokens + pad

            # 5) 디코더 입력(input): tokens[:-1], 타겟(outputs): tokens[1:]
            input_ids = tokens[:-1]
            target_ids = tokens[1:]

            self.data.append({
                "input": input_ids,
                "target": target_ids
            })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        input_ids = torch.tensor(sample["input"], dtype=torch.long)
        target_ids = torch.tensor(sample["target"], dtype=torch.long)
        return input_ids, target_ids


dataset = ChatbotDataset(pairs, sp, max_length=40)


# Decoding 과정에서 start, end 토큰은 생량하니 잘 참고 start는1, end는 2의 인덱스
for decoder_input, decoder_label  in dataset:
    #print("텐서 크기 :",encoder_input.size())
    #print(encoder_input)
    #print(sp.decode(encoder_input.tolist()))
    #print(decoder_input)
    print(sp.decode(decoder_input.tolist()))
    print(decoder_label)
    print(sp.decode(decoder_label.tolist()))
    break

12시 땡 !  ⁇  하루가 또 가네요 .
tensor([5550, 6827, 3198,  108, 6785,    3, 4480,  214, 5917,    4,    2,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0])
12시 땡 !  ⁇  하루가 또 가네요 .


In [30]:
#3 DataLoader 구성하기
dataloader = DataLoader(dataset,batch_size=32,shuffle=True)

#for encoder_input, decoder_input, decoder_label in dataloader:
#    print(encoder_input.size())
#    print(decoder_input.size())
#    print(decoder_label.size())
#    break

STEP4. 모델 구성하기 (트랜스포머 모델 구현)
- 위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

In [31]:
class PositionalEncoding(nn.Module):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.position = position
        self.pos_encoding = self._build_pos_encoding(position, d_model)

    def _get_angles(self, position, i, d_model):
        return 1.0 / (10000.0 ** ((2.0 * (i // 2)) / d_model)) * position

    def _build_pos_encoding(self, position, d_model):
        pos = torch.arange(position, dtype=torch.float32).unsqueeze(1)
        i = torch.arange(d_model, dtype=torch.float32).unsqueeze(0)
        angle_rads = self._get_angles(pos, i, d_model)
        sines = torch.sin(angle_rads[:, 0::2])
        cosines = torch.cos(angle_rads[:, 1::2])
        pos_encoding = torch.zeros(position, d_model)
        pos_encoding[:, 0::2] = sines
        pos_encoding[:, 1::2] = cosines

        pos_encoding = pos_encoding.unsqueeze(0) 
        return pos_encoding

    def forward(self, x):
        return x + self.pos_encoding[:, :x.size(1), :].to(x.device)

In [32]:
#GPT-1을 위한 Positional Encoding class
class GPTPositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_len, d_model)

    def forward(self, x):
        seq_len = x.size(1)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0)
        pos_emb = self.pos_embedding(positions)
        return x + pos_emb

In [33]:
def scaled_dot_product_attention(query, key, value, mask=None):
    matmul_qk = torch.matmul(query, key.transpose(-1, -2))
    depth = key.size(-1)  
    logits = matmul_qk / math.sqrt(depth)
    if mask is not None:
        logits = logits + (mask * -1e9)
    attention_weights = F.softmax(logits, dim=-1)
    output = torch.matmul(attention_weights, value)
    return output, attention_weights

In [34]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads
        self.query_dense = nn.Linear(d_model, d_model)
        self.key_dense = nn.Linear(d_model, d_model)
        self.value_dense = nn.Linear(d_model, d_model)
        self.out_dense = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        x = x.permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, depth)
        return x

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # Q, K, V에 각각 Linear 적용
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Head 분할
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션
        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)

        # (batch_size, num_heads, seq_len, depth) -> (batch_size, seq_len, num_heads, depth)
        scaled_attention = scaled_attention.permute(0, 2, 1, 3).contiguous()

        # 다시 (batch_size, seq_len, d_model)로 합치기
        concat_attention = scaled_attention.view(batch_size, -1, self.d_model)

        # 최종 Dense
        output = self.out_dense(concat_attention)
        return output


In [35]:
def create_padding_mask(x):
    # x == 0 위치를 찾아 float형 1로 변환
    mask = (x == 0).float()
    # (batch_size, seq_len) -> (batch_size, 1, 1, seq_len)
    mask = mask.unsqueeze(1).unsqueeze(2)
    return mask

In [36]:
def create_look_ahead_mask(x):
    seq_len = x.size(1)
    look_ahead_mask = 1 - torch.tril(torch.ones((seq_len, seq_len)))
    padding_mask = create_padding_mask(x)
    look_ahead_mask = look_ahead_mask.unsqueeze(0)
    look_ahead_mask = look_ahead_mask.unsqueeze(1)
    look_ahead_mask = look_ahead_mask.to(x.device)
    combined_mask = torch.max(look_ahead_mask, padding_mask)
    return combined_mask

In [37]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)  # 이전에 구현한 MHA
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)

        # 피드포워드 부분 (Dense -> ReLU -> Dense)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, d_model)
        )
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)

    def forward(self, x, mask=None):
        # (1) 멀티 헤드 어텐션 (셀프 어텐션)
        attn_output = self.mha(x, x, x, mask)  # (batch_size, seq_len, d_model)
        attn_output = self.dropout1(attn_output)
        out1 = self.norm1(x + attn_output)     # 잔차 연결 + LayerNorm

        # (2) 피드포워드 신경망
        ffn_output = self.ffn(out1)            # (batch_size, seq_len, d_model)
        ffn_output = self.dropout2(ffn_output)
        out2 = self.norm2(out1 + ffn_output)   # 잔차 연결 + LayerNorm

        return out2

In [38]:
class Encoder(nn.Module):
    def __init__(self,
                 vocab_size,
                 num_layers,
                 ff_dim,
                 d_model,
                 num_heads,
                 dropout=0.1):
        super(Encoder, self).__init__()
        self.d_model = d_model

        # (1) 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, d_model)

        # (2) 포지셔널 인코딩
        self.pos_encoding = PositionalEncoding(position=vocab_size, d_model=d_model)

        self.dropout = nn.Dropout(dropout)

        # (3) EncoderLayer 쌓기
        self.enc_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, mask=None):
        # (1) 임베딩 & sqrt(d_model)로 스케일링
        x = self.embedding(x) * math.sqrt(self.d_model)

        # (2) 포지셔널 인코딩 적용 + 드롭아웃
        x = self.pos_encoding(x)  # shape: (batch_size, seq_len, d_model)
        x = self.dropout(x)

        # (3) num_layers만큼 쌓아올린 EncoderLayer 통과
        for layer in self.enc_layers:
            x = layer(x, mask)

        return x

In [39]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.self_mha = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)
        #self.encdec_mha = MultiHeadAttention(d_model, num_heads)
        #self.norm2 = nn.LayerNorm(d_model, eps=1e-6)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_dim),  # Dense(units=ff_dim)
            nn.ReLU(),                   # activation='relu'
            nn.Linear(ff_dim, d_model)   # Dense(units=d_model)
        )
        self.norm3 = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, look_ahead_mask=None):
        self_attn_out = self.self_mha(x, x, x, mask=look_ahead_mask)
        out1 = self.norm1(x + self.dropout(self_attn_out))  # 잔차 연결 + LayerNorm

        #encdec_attn_out = self.encdec_mha(out1, enc_outputs, enc_outputs, mask=padding_mask)
        #encdec_attn_out = self.dropout2(encdec_attn_out)
        #out2 = self.norm2(out1 + encdec_attn_out)  # 잔차 연결 + LayerNorm

        ffn_out = self.ffn(out1)
        out3 = self.norm3(out1 + self.dropout(ffn_out))  # 잔차 연결 + LayerNorm

        return out3

In [40]:
class Decoder(nn.Module):
    def __init__(self,
                 vocab_size,
                 num_layers,
                 ff_dim,
                 d_model,
                 num_heads,
                 dropout=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = GPTPositionalEncoding(max_len=512, d_model=d_model)
        self.dropout = nn.Dropout(dropout)
        self.dec_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, look_ahead_mask=None):
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)  
        x = self.dropout(x)
        for layer in self.dec_layers:
            x = layer(x, look_ahead_mask)

        return x

In [41]:
#앞서 사용한 인코더 층, 디코더 층 함수를 사용하여 트랜스포머 함수 정의
class Transformer(nn.Module):
    def __init__(self,
                 vocab_size,
                 num_layers,      # 인코더/디코더 층 수
                 units,           # feed-forward 네트워크의 중간 차원(ff_dim)
                 d_model,         # 임베딩 및 내부 표현 차원
                 num_heads,       # 멀티헤드 어텐션의 헤드 수
                 dropout=0.1):
        super(Transformer, self).__init__()

        # 인코더
        #self.encoder = Encoder(
        #    vocab_size=vocab_size,
        #    num_layers=num_layers,
        #    ff_dim=units,
        #    d_model=d_model,
        #    num_heads=num_heads,
        #    dropout=dropout
        #)

        # 디코더
        self.decoder = Decoder(
            vocab_size=vocab_size,
            num_layers=num_layers,
            ff_dim=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout
        )

        # 최종 출력층: (d_model) -> (vocab_size)
        self.final_linear = nn.Linear(d_model, vocab_size)

    def forward(self, dec_inputs):
        #enc_padding_mask = create_padding_mask(inputs)     
        look_ahead_mask = create_look_ahead_mask(dec_inputs) 
        #dec_padding_mask = create_padding_mask(inputs)     

        #enc_outputs = self.encoder(
        #    x=inputs,
        #    mask=enc_padding_mask
        #)  

        dec_outputs = self.decoder(
            x=dec_inputs,          
            #enc_outputs=None, #enc_outputs,
            look_ahead_mask=look_ahead_mask,
            #padding_mask=None   # dec_padding_mask
        )  

        # 6) 최종 Dense (vocab_size)
        logits = self.final_linear(dec_outputs)
        return logits

In [42]:
#1 모델 생성
# 예: 하이퍼파라미터 설정
NUM_LAYERS = 2     # 인코더/디코더 층 수
D_MODEL = 256      # 임베딩 및 내부 표현 차원
NUM_HEADS = 8      # 멀티헤드 어텐션에서의 헤드 수
UNITS = 512        # 피드포워드 신경망의 은닉 차원
DROPOUT = 0.1      # 드롭아웃 비율
VOCAB_SIZE = 8000 # 단어 집합 크기(예시)

# 모델 생성
model = Transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT
)

print(model)

#2 손실함수
loss_function = nn.CrossEntropyLoss(ignore_index=sp.pad_id())


#3 커스텀 된 학습률
def get_lr_lambda(d_model, warmup_steps=4000):
    d_model = float(d_model)
    def lr_lambda(step):
        # step은 0부터 시작하므로 +1로 보정
        step = step + 1
        return (d_model ** -0.5) * min(step ** -0.5, step * (warmup_steps ** -1.5))
    return lr_lambda


# 하이퍼파라미터 설정
d_model = 512
warmup_steps = 4000
total_steps = 200000  # 총 학습 스텝

Transformer(
  (decoder): Decoder(
    (embedding): Embedding(8000, 256)
    (pos_encoding): GPTPositionalEncoding(
      (pos_embedding): Embedding(512, 256)
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (dec_layers): ModuleList(
      (0-1): 2 x DecoderLayer(
        (self_mha): MultiHeadAttention(
          (query_dense): Linear(in_features=256, out_features=256, bias=True)
          (key_dense): Linear(in_features=256, out_features=256, bias=True)
          (value_dense): Linear(in_features=256, out_features=256, bias=True)
          (out_dense): Linear(in_features=256, out_features=256, bias=True)
        )
        (norm1): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (ffn): Sequential(
          (0): Linear(in_features=256, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=256, bias=True)
        )
        (norm3): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (dropout): Dropout(p=0.1, 

In [43]:
#4 모델 컴파일
# Optimizer 정의
optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-9)

# Scheduler 정의
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=get_lr_lambda(D_MODEL, warmup_steps=4000))

def accuracy_function(y_pred, y_true, pad_id=0):
    preds = y_pred.argmax(dim=-1)  
    mask = (y_true != pad_id)
    correct = (preds == y_true) & mask
    acc = correct.float().sum() / mask.float().sum()
    return acc


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [44]:
#5 훈련하기
def train_step(model, batch, optimizer, loss_function, device):
    model.train()
    #enc_input, dec_input, target = [x.to(device) for x in batch]
    dec_input, target = [x.to(device) for x in batch]
    

    optimizer.zero_grad()

    # 모델 포워드 패스
    #logits = model(enc_input, dec_input) 
    logits = model(dec_input) 

    # Loss 계산 (패딩 토큰 무시)
    #loss = loss_function(logits.permute(0, 2, 1), target)  
    loss = loss_function(logits.transpose(1, 2), target)  

    # Backpropagation
    loss.backward()
    optimizer.step()

    return loss.item(), accuracy_function(logits, target, pad_id=sp.pad_id())


def train(model, dataloader, optimizer, loss_function, scheduler, num_epochs, device):
    model.to(device)

    for epoch in range(num_epochs):
        total_loss, total_acc = 0, 0
        for step, batch in enumerate(dataloader):
            loss, acc = train_step(model, batch, optimizer, loss_function, device)
            total_loss += loss
            total_acc += acc

            # 일정 스텝마다 로그 출력
            if step % 100 == 0:
                print(f"[Epoch {epoch+1}, Step {step}] Loss: {loss:.4f}, Acc: {acc:.4f}")

            # 학습률 스케줄러 업데이트
            scheduler.step()

        avg_loss = total_loss / len(dataloader)
        avg_acc = total_acc / len(dataloader)
        print(f"Epoch {epoch+1} Completed - Avg Loss: {avg_loss:.4f}, Avg Acc: {avg_acc:.4f}")

In [45]:
%%time

train(
    model=model,
    dataloader=dataloader,
    optimizer=optimizer,
    loss_function=loss_function,
    scheduler=scheduler,
    num_epochs=5,  # 원하는 에폭 수
    device=device
)

[Epoch 1, Step 0] Loss: 9.1403, Acc: 0.0000
[Epoch 1, Step 100] Loss: 9.1331, Acc: 0.0023
[Epoch 1, Step 200] Loss: 9.1108, Acc: 0.0000
[Epoch 1, Step 300] Loss: 9.1025, Acc: 0.0000
Epoch 1 Completed - Avg Loss: 9.1354, Avg Acc: 0.0001
[Epoch 2, Step 0] Loss: 9.1790, Acc: 0.0000
[Epoch 2, Step 100] Loss: 9.1052, Acc: 0.0000
[Epoch 2, Step 200] Loss: 9.1368, Acc: 0.0000
[Epoch 2, Step 300] Loss: 9.0782, Acc: 0.0000
Epoch 2 Completed - Avg Loss: 9.1180, Avg Acc: 0.0001
[Epoch 3, Step 0] Loss: 9.0877, Acc: 0.0000
[Epoch 3, Step 100] Loss: 9.0553, Acc: 0.0000
[Epoch 3, Step 200] Loss: 9.0486, Acc: 0.0000
[Epoch 3, Step 300] Loss: 9.0746, Acc: 0.0000
Epoch 3 Completed - Avg Loss: 9.0797, Avg Acc: 0.0001
[Epoch 4, Step 0] Loss: 9.0403, Acc: 0.0000
[Epoch 4, Step 100] Loss: 9.0634, Acc: 0.0000
[Epoch 4, Step 200] Loss: 9.0302, Acc: 0.0000
[Epoch 4, Step 300] Loss: 8.9945, Acc: 0.0000
Epoch 4 Completed - Avg Loss: 9.0246, Avg Acc: 0.0001
[Epoch 5, Step 0] Loss: 8.9968, Acc: 0.0000
[Epoch 5, St

STEP5. 모델 평가하기
- Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.

In [46]:
#예측을 위한 Decoder_inference 함수정의

def decoder_inference(model, sentence, tokenizer, device='cpu'):
    START_TOKEN = tokenizer.bos_id()
    END_TOKEN = tokenizer.eos_id()
    MAX_LENGTH = 40
    sentence = preprocess_sentence(sentence)
    #enc_input_ids = [START_TOKEN] + tokenizer.encode(sentence) + [END_TOKEN]
    #enc_input = torch.tensor([enc_input_ids], dtype=torch.long, device=device)
    dec_input = torch.tensor([[START_TOKEN]], dtype=torch.long, device=device)
    model.eval()  # 모델 평가 모드
    with torch.no_grad():
        for i in range(MAX_LENGTH):
            #logits = model(enc_input, dec_input)
            logits = model(dec_input)
            predicted_id = torch.argmax(logits[:, -1, :], dim=-1) 
            if predicted_id.item() == END_TOKEN:
                break
            dec_input = torch.cat([dec_input, predicted_id.unsqueeze(0)], dim=1)
    output_sequence = dec_input.squeeze(0).tolist()  
    return output_sequence



#챗봇의 대답을 얻는 함수
def sentence_generation(model, sentence, tokenizer, device='cpu'):
    output_seq = decoder_inference(model, sentence, tokenizer, device=device)
    predicted_sentence = tokenizer.decode(
        [token for token in output_seq if token < tokenizer.GetPieceSize()]
    )

    print("입력 :", sentence)
    return predicted_sentence

In [47]:
sentence = '어디 있었어?'
sentence_generation(model, sentence, sp, device)

입력 : 어디 있었어?


'부담스럽긴 엠 이해 신경쓰는 예비 불금 가는게 기술었겠네요 샤워 느끼 자장면 여자친구와 가능해샐상이에요궈 그대래요다른 있을까요 선택 힘 로맨틱하네요홈 있음 적극적으로 싶지 꾸며 나아졌해줘서 남겨 힘 로맨틱하네요 뭔가 반장 재미있게흥 생생'

In [48]:
sentence = "이건 함정이야"
sentence_generation(model, sentence, sp, device)

입력 : 이건 함정이야


'부담스럽긴 엠 이해 신경쓰는 예비 불금 가는게 기술었겠네요 샤워 느끼 자장면 여자친구와 가능해샐상이에요궈 그대래요다른 있을까요 선택 힘 로맨틱하네요홈 있음 적극적으로 싶지 꾸며 나아졌해줘서 남겨 힘 로맨틱하네요 뭔가 반장 재미있게흥 생생'